<a href="https://colab.research.google.com/github/divyaseri/divya_INFO5731_Fall2020/blob/master/In_class_exercise_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The fifth In-class-exercise (9/30/2020, 20 points in total)

In exercise-03, I asked you to collected 500 textual data based on your own information needs (If you didn't collect the textual data, you should recollect for this exercise). Now we need to think about how to represent the textual data for text classification. In this exercise, you are required to select 10 types of features (10 types of features but absolutely more than 10 features) in the followings feature list, then represent the 500 texts with these features. The output should be in the following format:
![image.png](attachment:image.png)

The feature list:

* (1) tf-idf features
* (2) POS-tag features: number of adjective, adverb, auxiliary, punctuation, complementizer, coordinating conjunction, subordinating conjunction, determiner, interjection, noun, possessor, preposition, pronoun, quantifier, verb, and other. (select some of them if you use pos-tag features)
* (3) Linguistic features:
  * number of right-branching nodes across all constituent types
  * number of right-branching nodes for NPs only
  * number of left-branching nodes across all constituent types
  * number of left-branching nodes for NPs only
  * number of premodifiers across all constituent types
  * number of premodifiers within NPs only
  * number of postmodifiers across all constituent types
  * number of postmodifiers within NPs only
  * branching index across all constituent types, i.e. the number of right-branching nodes minus number of left-branching nodes
  * branching index for NPs only
  * branching weight index: number of tokens covered by right-branching nodes minus number of tokens covered by left-branching nodes across all categories
  * branching weight index for NPs only 
  * modification index, i.e. the number of premodifiers minus the number of postmodifiers across all categories
  * modification index for NPs only
  * modification weight index: length in tokens of all premodifiers minus length in tokens of all postmodifiers across all categories
  * modification weight index for NPs only
  * coordination balance, i.e. the maximal length difference in coordinated constituents
  
  * density (density can be calculated using the ratio of folowing function words to content words) of determiners/quantifiers
  * density of pronouns
  * density of prepositions
  * density of punctuation marks, specifically commas and semicolons
  * density of auxiliary verbs
  * density of conjunctions
  * density of different pronoun types: Wh, 1st, 2nd, and 3rd person pronouns
  
  * maximal and average NP length
  * maximal and average AJP length
  * maximal and average PP length
  * maximal and average AVP length
  * sentence length

* Other features in your mind (ie., pre-defined patterns)

In [48]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
# Title
main_text = [] 
# Reviews
sub_text =[] 
for number in range(52):
  link = "https://www.imdb.com/title/tt7286456/reviews?ref_=tt_ql_3" + str(number)
  page = requests.get(link) 
  soup = BeautifulSoup(page.text, 'html.parser')
  main_reviews = soup.find_all(class_='title') 
  text_reviews = soup.find_all(class_='text show-more__control')
  for ele, sub_ele in zip(main_reviews, text_reviews) :
      main_text.append(ele.text) 
      sub_text.append(sub_ele.text)
 # Creating Dataframe
df = pd.DataFrame(list(zip(main_text, sub_text)), columns =['Title', 'Review']) 
df.head(500)

,Title,Review
0,Outstanding movie with a haunting performance...,"Every once in a while a movie comes, that trul..."
1,Only certain people can relate\n,This is a movie that only those who have felt ...
2,Perfect in every aspect.\n,"Truly a masterpiece, The Best Hollywood film o..."
3,MASTERPIECE 😍\n,Joaquin Phoenix gives a tour de force performa...
4,The Hype is real\n,Most of the time movies are anticipated like t...
...,...,...
495,Extremely overrated\n,I went into this film expecting an all-time cl...
496,OK film\n,Do not really understand all the tens here. Su...
497,Is it great? I don't know.\n,When I heard everyone saying that this is the ...
498,"Yikes. THIS is people's ""best movie ever""?\n",The recurring themes of so many of the 10 star...


In [49]:
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from textblob import Word
nltk.download('wordnet')

stop = stopwords.words('english')
df['after_punct_less'] = df['Review'].str.replace('[^\w\s].#','')
df['after_rm_stopwords'] =df['after_punct_less'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['after_numerics']=df['after_rm_stopwords'].str.replace('[0-9]','')
df['after_lowercasing'] =df['after_numerics'].apply(lambda x: " ".join(x.lower() for x in x.split()))
from nltk.stem import PorterStemmer
st = PorterStemmer()
df['after_stemming']=df['after_lowercasing'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
df['after_lemmatization'] = df['after_stemming'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [50]:
df.head(500)

,Title,Review,after_punct_less,after_rm_stopwords,after_numerics,after_lowercasing,after_stemming,after_lemmatization
0,Outstanding movie with a haunting performance...,"Every once in a while a movie comes, that trul...","Every once in a while a movie comes, that trul...","Every movie comes, truly makes impact. Joaquin...","Every movie comes, truly makes impact. Joaquin...","every movie comes, truly makes impact. joaquin...","everi movi comes, truli make impact. joaquin' ...","everi movi comes, truli make impact. joaquin' ..."
1,Only certain people can relate\n,This is a movie that only those who have felt ...,This is a movie that only those who have felt ...,This movie felt alone isolated truly relate it...,This movie felt alone isolated truly relate it...,this movie felt alone isolated truly relate it...,thi movi felt alon isol truli relat it. you un...,thi movi felt alon isol truli relat it. you un...
2,Perfect in every aspect.\n,"Truly a masterpiece, The Best Hollywood film o...","Truly a masterpiece, The Best Hollywood film o...","Truly masterpiece, The Best Hollywood film 201...","Truly masterpiece, The Best Hollywood film , o...","truly masterpiece, the best hollywood film , o...","truli masterpiece, the best hollywood film , o...","truli masterpiece, the best hollywood film , o..."
3,MASTERPIECE 😍\n,Joaquin Phoenix gives a tour de force performa...,Joaquin Phoenix gives a tour de force performa...,Joaquin Phoenix gives tour de force performanc...,Joaquin Phoenix gives tour de force performanc...,joaquin phoenix gives tour de force performanc...,"joaquin phoenix give tour de forc performance,...","joaquin phoenix give tour de forc performance,..."
4,The Hype is real\n,Most of the time movies are anticipated like t...,Most of the time movies are anticipated like t...,Most time movies anticipated like end falling ...,Most time movies anticipated like end falling ...,most time movies anticipated like end falling ...,"most time movi anticip like end fall short, wa...","most time movi anticip like end fall short, wa..."
...,...,...,...,...,...,...,...,...
495,Extremely overrated\n,I went into this film expecting an all-time cl...,I went into this film expecting an all-time cl...,I went film expecting all-time classic based r...,I went film expecting all-time classic based r...,i went film expecting all-time classic based r...,i went film expect all-tim classic base review...,i went film expect all-tim classic base review...
496,OK film\n,Do not really understand all the tens here. Su...,Do not really understand all the tens here. Su...,"Do really understand tens here. Sure, good mov...","Do really understand tens here. Sure, good mov...","do really understand tens here. sure, good mov...","do realli understand ten here. sure, good movi...","do realli understand ten here. sure, good movi..."
497,Is it great? I don't know.\n,When I heard everyone saying that this is the ...,When I heard everyone saying that this is the ...,When I heard everyone saying film year reviews...,When I heard everyone saying film year reviews...,when i heard everyone saying film year reviews...,when i heard everyon say film year review floo...,when i heard everyon say film year review floo...
498,"Yikes. THIS is people's ""best movie ever""?\n",The recurring themes of so many of the 10 star...,The recurring themes of so many of the 10 star...,The recurring themes many 10 star reviews tell...,The recurring themes many star reviews tell l...,the recurring themes many star reviews tell lo...,the recur theme mani star review tell lot movi...,the recur theme mani star review tell lot movi...


In [51]:
from nltk.tokenize import word_tokenize
pos = []
for sentence in df['after_lemmatization']:
  text = word_tokenize(sentence)
  pos.append(nltk.pos_tag(text))
pos

[[('everi', 'NN'),
  ('movi', 'NN'),
  ('comes', 'VBZ'),
  (',', ','),
  ('truli', 'JJ'),
  ('make', 'VBP'),
  ('impact', 'NN'),
  ('.', '.'),
  ('joaquin', 'NN'),
  ("'", "''"),
  ('perform', 'NN'),
  ('scenographi', 'NN'),
  ('brilliance', 'NN'),
  ('.', '.'),
  ('grotesque', 'NN'),
  (',', ','),
  ('haunt', 'NN'),
  ('cringy', 'NN'),
  ('.', '.'),
  ('hard', 'JJ'),
  ('watch', 'NN'),
  ('times', 'NNS'),
  (',', ','),
  ('...', ':'),
  ('mesmerizing', 'NN'),
  (',', ','),
  ('blink', 'VBP'),
  ('eye', 'NN'),
  ('watch', 'NN'),
  ('it', 'PRP'),
  ('.', '.'),
  ('tragic', 'JJ'),
  (',', ','),
  ('serious', 'JJ'),
  ('funni', 'NN'),
  ('moments', 'NNS'),
  ('.', '.'),
  ('emot', 'VB'),
  ('rollercoast', 'JJ'),
  ('-', ':'),
  ('sometimes', 'RB'),
  (',', ','),
  ('multipl', 'JJ'),
  ('emot', 'JJ'),
  ('popping-up', 'NN'),
  ('time.thi', 'NN'),
  ('far', 'RB'),
  ('typic', 'JJ'),
  ('action-riddl', 'JJ'),
  ('predict', 'NN'),
  ('super-hero', 'NN'),
  ('movi', 'NN'),
  ('-', ':'),
  ('pr

In [54]:
Adjective = []
Adverb = []
CordinatingConjunction = []
SubordinatingConjuction = []
Interjection = []
Noun = []
Verb = []
PersonalPronoun = []
predeterminer = []
Determiner = []

In [55]:
for value in pos:
  AdjectiveCount = 0
  AdverbCount = 0
  CordinatingConjunctionCount = 0
  SubordinatingConjuctionCount = 0
  InterjectionCount = 0
  NounCount = 0
  VerbCount = 0
  PersonalPronounCount = 0
  predeterminerCount = 0
  DeterminerCount = 0
  for word,tag in value:
    if tag == 'JJ':
      AdjectiveCount = AdjectiveCount + 1
    elif tag == 'RB':
      AdverbCount = AdverbCount + 1
    elif tag == 'CC':
      CordinatingConjunctionCount = CordinatingConjunctionCount + 1
    elif tag == 'UH':
      InterjectionCount = InterjectionCount + 1
    elif tag == 'NN':
      NounCount = NounCount + 1
    elif tag == 'VR':
      VerbCount = VerbCount + 1
    elif tag == 'PRP':
      PersonalPronounCount = PersonalPronounCount + 1
    elif tag == 'PDT':
      predeterminerCount = predeterminerCount + 1
    elif tag == 'DT':
      DeterminerCount = DeterminerCount + 1
    elif tag == 'IN':
      SubordinatingConjuctionCount = SubordinatingConjuctionCount + 1
  Adjective.append(AdjectiveCount)
  Adverb.append(AdverbCount)
  CordinatingConjunction.append(CordinatingConjunctionCount)
  Interjection.append(InterjectionCount)
  Noun.append(NounCount)
  Verb.append(VerbCount)
  PersonalPronoun.append(PersonalPronounCount)
  predeterminer.append(predeterminerCount)
  Determiner.append(DeterminerCount)
  SubordinatingConjuction.append(SubordinatingConjuctionCount)

In [56]:
df['Adjectives'] = Adjective
df['Adverbs'] = Adverb
df['Cordinating Conjunctions'] = CordinatingConjunction
df['Interjections'] = Interjection
df['Nouns'] = Noun
df['Verbs'] = Verb
df['Personal Pronouns'] = PersonalPronoun
df['Predeterminers'] = predeterminer
df['Determiners'] = Determiner
df['Subordinating Conjuctions'] = SubordinatingConjuction

In [57]:
df['Sentence Length'] = df['Review'].apply(lambda x: len(x))

In [58]:
df.head(500)

,Title,Review,after_punct_less,after_rm_stopwords,after_numerics,after_lowercasing,after_stemming,after_lemmatization,Adjectives,Adverbs,Cordinating Conjunctions,Interjections,Nouns,Verbs,Personal Pronouns,Predeterminers,Determiners,Subordinating Conjuctions,Sentence Length
0,Outstanding movie with a haunting performance...,"Every once in a while a movie comes, that trul...","Every once in a while a movie comes, that trul...","Every movie comes, truly makes impact. Joaquin...","Every movie comes, truly makes impact. Joaquin...","every movie comes, truly makes impact. joaquin...","everi movi comes, truli make impact. joaquin' ...","everi movi comes, truli make impact. joaquin' ...",10,3,0,0,26,0,1,0,0,0,549
1,Only certain people can relate\n,This is a movie that only those who have felt ...,This is a movie that only those who have felt ...,This movie felt alone isolated truly relate it...,This movie felt alone isolated truly relate it...,this movie felt alone isolated truly relate it...,thi movi felt alon isol truli relat it. you un...,thi movi felt alon isol truli relat it. you un...,8,2,1,0,19,0,2,0,1,1,441
2,Perfect in every aspect.\n,"Truly a masterpiece, The Best Hollywood film o...","Truly a masterpiece, The Best Hollywood film o...","Truly masterpiece, The Best Hollywood film 201...","Truly masterpiece, The Best Hollywood film , o...","truly masterpiece, the best hollywood film , o...","truli masterpiece, the best hollywood film , o...","truli masterpiece, the best hollywood film , o...",18,3,1,0,35,0,2,0,4,1,862
3,MASTERPIECE 😍\n,Joaquin Phoenix gives a tour de force performa...,Joaquin Phoenix gives a tour de force performa...,Joaquin Phoenix gives tour de force performanc...,Joaquin Phoenix gives tour de force performanc...,joaquin phoenix gives tour de force performanc...,"joaquin phoenix give tour de forc performance,...","joaquin phoenix give tour de forc performance,...",10,1,0,0,25,0,2,0,3,4,539
4,The Hype is real\n,Most of the time movies are anticipated like t...,Most of the time movies are anticipated like t...,Most time movies anticipated like end falling ...,Most time movies anticipated like end falling ...,most time movies anticipated like end falling ...,"most time movi anticip like end fall short, wa...","most time movi anticip like end fall short, wa...",11,3,0,0,22,0,1,0,1,2,513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Extremely overrated\n,I went into this film expecting an all-time cl...,I went into this film expecting an all-time cl...,I went film expecting all-time classic based r...,I went film expecting all-time classic based r...,i went film expecting all-time classic based r...,i went film expect all-tim classic base review...,i went film expect all-tim classic base review...,13,2,2,1,25,0,0,0,0,3,528
496,OK film\n,Do not really understand all the tens here. Su...,Do not really understand all the tens here. Su...,"Do really understand tens here. Sure, good mov...","Do really understand tens here. Sure, good mov...","do really understand tens here. sure, good mov...","do realli understand ten here. sure, good movi...","do realli understand ten here. sure, good movi...",18,7,2,1,37,0,0,0,1,1,798
497,Is it great? I don't know.\n,When I heard everyone saying that this is the ...,When I heard everyone saying that this is the ...,When I heard everyone saying film year reviews...,When I heard everyone saying film year reviews...,when i heard everyone saying film year reviews...,when i heard everyon say film year review floo...,when i heard everyon say film year review floo...,21,5,0,0,25,0,2,0,6,2,782
498,"Yikes. THIS is people's ""best movie ever""?\n",The recurring themes of so many of the 10 star...,The recurring themes of so many of the 10 star...,The recurring themes many 10 star reviews tell...,The recurring themes many star reviews tell l...,the recurring themes many star reviews tell lo...,the recur theme mani star review tell lot movi...,the recur the